[![Binder](https://mybinder.org/badge_logo.svg)](https://nbviewer.org/github/vicente-gonzalez-ruiz/color_transforms/blob/main/docs/YUV_quantization.ipynb)

[![Colab](https://badgen.net/badge/Launch/on%20Google%20Colab/blue?icon=notebook)](https://colab.research.google.com/github/vicente-gonzalez-ruiz/color_transforms/blob/main/docs/YUV_quantization.ipynb)

# YUV image compression

In [ ]:
import math
import os

In [ ]:
try:
    import matplotlib
    import matplotlib.pyplot as plt
    import matplotlib.axes as ax
    #plt.rcParams['text.usetex'] = True
    #plt.rcParams['text.latex.preamble'] = [r'\usepackage{amsmath}'] #for \text command
    import pylab
except:
    !pip install matplotlib
    import matplotlib
    import matplotlib.pyplot as plt
    import matplotlib.axes as ax
    #plt.rcParams['text.usetex'] = True
    #plt.rcParams['text.latex.preamble'] = [r'\usepackage{amsmath}'] #for \text command
    import pylab

%matplotlib inline

In [ ]:
try:
    import numpy as np
except:
    !pip install numpy
    import numpy as np 

In [ ]:
try:
    from scipy import signal
except:
    !pip install scipy
    from scipy import signal

In [ ]:
try:
    import cv2
except:
    !pip install opencv-python
    import cv2

In [ ]:
try:
    import colored
except:
    !pip install colored
    import colored

In [ ]:
try:
    from skimage import io
except:
    !pip install scikit-image
    from skimage import io

In [ ]:
try:
    from scalar_quantization.deadzone_quantization import Deadzone_Quantizer as Quantizer                         
except:
    !pip install "scalar_quantization @ git+https://github.com/vicente-gonzalez-ruiz/scalar_quantization"
    from scalar_quantization.deadzone_quantization import Deadzone_Quantizer as Quantizer

In [ ]:
try:
    #from color_transforms.YCoCg import from_RGB
    #from color_transforms.YCoCg import to_RGB
    #from color_transforms.YCoCg import name as YUV_name
    from color_transforms import YCrCb
    from color_transforms import YCoCg
except:
    !pip install "color_transforms @ git+https://github.com/vicente-gonzalez-ruiz/color_transforms"
    #from color_transforms.YCoCg import from_RGB
    #from color_transforms.YCoCg import to_RGB
    #from color_transforms.YCoCg import name as YUV_name
    from color_transforms import YCrCb
    from color_transforms import YCoCg

In [ ]:
try:
    from information_theory import information
    from information_theory import distortion
except:
    !pip install "information_theory @ git+https://github.com/vicente-gonzalez-ruiz/information_theory"
    from information_theory import information
    from information_theory import distortion

In [ ]:
try:
    from image_IO import image_1 as gray_img
    from image_IO import image_3 as RGB_img
except:
    !pip install "image_IO @ git+https://github.com/vicente-gonzalez-ruiz/image_IO"
    from image_IO import image_1 as gray_img
    from image_IO import image_3 as RGB_img

!ln -sf ~/MRVC/src/frame.py .
!ln -sf ~/MRVC/src/debug.py
!ln -sf ~/MRVC/src/image_3.py
!ln -sf ~/MRVC/src/image_1.py
import deadzone as Q
import YCrCb
import YCoCg
import distortion
import information
import image_3 as RGB_img
import image_1 as gray_img
import colored

## Configuration area

In [ ]:
fn = "http://www.hpca.ual.es/~vruiz/images/lena.png"
Q_steps = 8
components = ['R', 'G', 'B']

In [ ]:
# Read the image and show it.

#img = RGB_img.read(fn)
img = io.imread(fn).astype(np.int16)
print(img.max(), img.min())

def show(img, title):
    img = RGB_img.normalize(img)
    plt.figure(figsize=(10,10))
    plt.title(title, fontsize=20)
    plt.imshow(img)

show(img, fn + "000.png")

In [ ]:
# Dead-zone quantizer & dequantizer.
# 
# Notice that, although this is a dead-zone quantizer, we are not going
# to work with negative samples, and therefore, the dead-zone
# does not have any effect.


def q_deq(x, quantization_step):
    Q = Quantizer(Q_step=quantization_step)
    k = Q.encode(x)
    y = Q.decode(k)
    return k, y

In [ ]:
def show_gray(img, title):
    img = gray_img.normalize(img)
    plt.figure(figsize=(10,10))
    plt.title(title, fontsize=20)
    plt.imshow(img, cmap='gray')
show_gray(img[:,:,0], "stockholm (R component)")
show_gray(img[:,:,1], "stockholm0000 (G component)")
show_gray(img[:,:,2], "stockholm000 (B component)")

## YCrCb components

In [ ]:
YCrCb_img = YCrCb.from_RGB(img.astype(np.uint8))
show_gray(YCrCb_img[:,:,0], "stockholm000 (Y component, YCrCb domain)")
show_gray(YCrCb_img[:,:,1], "stockholm000 (Cr component, YCrCb domain)")
show_gray(YCrCb_img[:,:,2], "stockholm000 (Cb component)")

### Is the YCrCb reversible?

In [ ]:
print(YCrCb_img.shape)
img2 = YCrCb.to_RGB(YCrCb_img)
print(np.array_equal(img, img2))
print(img.max(), img.min())
print(YCrCb_img.max(), YCrCb_img.min(), np.average(YCrCb_img))
show(img, "stockholm000 (original)")
show(img2, "stockholm000 (reconstructed through YCoCg)")
show(img-img2, 'stockholm000 - YCrCb.to\_RGB(YCrCb.from\_RGB(stockholm000))')

## YCoCg components

In [ ]:
YCoCg_img = YCoCg.from_RGB(img.astype(np.int16))
show_gray(YCoCg_img[:,:,0], "stockholm000 (Y component)")
show_gray(YCoCg_img[:,:,1], "stockholm000 (Co component)")
show_gray(YCoCg_img[:,:,2], "stockholm000 (Cg component)")

### Is the YCoCg reversible?

In [ ]:
img2 = YCoCg.to_RGB(YCoCg_img)
print(np.array_equal(img, img2))
print(img.max(), img.min())
print(YCoCg_img.max(), YCoCg_img.min(), np.average(YCoCg_img))
show(img, "stockholm000 (original)")
show(img2, "stockholm000 (reconstructed through YCoCg)")
show(img-img2, 'stockholm000 - YCoCg.to\_RGB(YCoCg.from\_RGB(stockholm000))')

## Gains of the YCrCb synthesis filters 

In [ ]:
val = 10
delta_YCrCb = np.array([val, 0, 0]).astype(np.uint8).reshape(1,1,3)
delta_RGB = YCrCb.to_RGB(delta_YCrCb)
print("delta_YCrCb =", delta_YCrCb, "delta_RGB =", delta_RGB, "Y gain =", information.average_energy(delta_RGB))

delta_YCrCb = np.array([0, val, 0]).astype(np.uint8).reshape(1,1,3)
delta_RGB = YCrCb.to_RGB(delta_YCrCb)
print("delta_YCrCb =", delta_YCrCb, "delta_RGB =", delta_RGB, "Cr gain =", information.average_energy(delta_RGB))

delta_YCrCb = np.array([0, 0, val]).astype(np.uint8).reshape(1,1,3)
delta_RGB = YCrCb.to_RGB(delta_YCrCb)
print("delta_YCrCb =", delta_YCrCb, "delta_RGB =", delta_RGB, "Cb gain =", information.average_energy(delta_RGB))

delta_YCrCb = np.array([val, val, val]).astype(np.uint8).reshape(1,1,3)
delta_RGB = YCrCb.to_RGB(delta_YCrCb)
print("delta_YCrCb =", delta_YCrCb, "delta_RGB =", delta_RGB, "Total gain =", information.average_energy(delta_RGB))

As we can see, the energy of the components in the YCrCb domain is not additive (the same happens with the distortion generated by the quantization).

In [ ]:
val = 10
delta_YCoCg = np.array([val, 0, 0]).reshape(1,1,3)
delta_RGB = YCoCg.to_RGB(delta_YCoCg)
print("delta_YCoCg =", delta_YCoCg, "delta_RGB =", delta_RGB, "Y gain =", information.average_energy(delta_RGB))

delta_YCoCg = np.array([0, val, 0]).reshape(1,1,3)
delta_RGB = YCoCg.to_RGB(delta_YCoCg)
print("delta_YCoCg =", delta_YCoCg, "delta_RGB =", delta_RGB, "Cr gain =", information.average_energy(delta_RGB))

delta_YCoCg = np.array([0, 0, val]).reshape(1,1,3)
delta_RGB = YCoCg.to_RGB(delta_YCoCg)
print("delta_YCoCg =", delta_YCoCg, "delta_RGB =", delta_RGB, "Cb gain =", information.average_energy(delta_RGB))

delta_YCoCg = np.array([val, val, val]).reshape(1,1,3)
delta_RGB = YCoCg.to_RGB(delta_YCoCg)
print("delta_YCoCg =", delta_YCoCg, "delta_RGB =", delta_RGB, "Total gain =", information.average_energy(delta_RGB))

With the YCoCg happens the same (the energy of the components is not additive). The gains matches with the theory.

### Energy of the RGB channels

In [ ]:
R_energy = information.average_energy(img[:,:,0])
G_energy = information.average_energy(img[:,:,1])
B_energy = information.average_energy(img[:,:,2])
print("Energy of R =", R_energy)
print("Energy of G =", G_energy)
print("Energy of B =", B_energy)

### Energy of the YCrCb channels

In [ ]:
YCrCb_frame = YCrCb.from_RGB(img.astype(np.uint8))
Y_energy = information.average_energy(YCrCb_frame[:,:,0])
Cr_energy = information.average_energy(YCrCb_frame[:,:,1])
Cb_energy = information.average_energy(YCrCb_frame[:,:,2])
print("Energy of Y =", Y_energy)
print("Energy of Cr =", Cr_energy)
print("Energy of Cb =", Cb_energy)

### Energy of the YCoCg channels

In [ ]:
YCoCg_frame = YCoCg.from_RGB(img.astype(np.int16))
Y_energy = information.average_energy(YCoCg_frame[:,:,0])
Co_energy = information.average_energy(YCoCg_frame[:,:,1])
Cg_energy = information.average_energy(YCoCg_frame[:,:,2])
print("Energy of Y =", Y_energy)
print("Energy of Co =", Co_energy)
print("Energy of Cg =", Cg_energy)

The energy is more concentrated in the YCoCg domain, more specifically in the Y channel.

## Generation of the optimal RD curve

The optimal RD curve can be generated with:

1. Convert the image from RGB to YUV.
2. The RD curve of each YUV channel is computed, for a number of quantization steps.
3. The OTP (Optimal Truncation Points) are sorted by slopes.

## Some helper functions

In [ ]:
# Number of bytes that a img "img" requires in disk.
def bytes_per_RGB_img(img, fn="img"):
    RGB_img.write(img, "/tmp/" + fn + ".png")
    length_in_bytes = os.path.getsize("/tmp/" + fn + ".png")
    return length_in_bytes

# The same value, but in bits/pixel.
def bits_per_RGB_pixel(img, fn="img"):
    return 8*bytes_per_RGB_img(img, fn)/(img.shape[0]*img.shape[1])

# Specific version for grayscale images.
def bytes_per_gray_img(img, fn="img"):
    cv2.imwrite("/tmp/" + fn + ".png", img)
    length_in_bytes = os.path.getsize("/tmp/" + fn + ".png")
    print(colored.fore.GREEN + f"bytes_per_gray_img: /tmp/{fn}.png", img.shape, img.dtype, length_in_bytes, colored.style.RESET)
    return length_in_bytes

# The same value, but in bits/pixel.
def bits_per_gray_pixel(img, fn="img"):
    return 8*bytes_per_gray_img(img, fn)/(img.shape[0]*img.shape[1])

## Build the optimal RD curve

Neither YCrCb nor YCoCg are orthogonal spaces, which means that even considering the channel gains, the contributions of the channels to the quality of the reconstruction are not additive (the channels are not independent). This implies that the search of the optimal QSs cannot be done by simply sorting the slopes of each OTP of each channel, but by searching for each OTP the best combination of QSs, that not necessarily need to be embbeded (i.e., it can not be asserted that the optimal QSs for a quality level Q generated by a combination $[\Delta_Y^Q, \Delta_U^Q, \Delta_V^Q]$ > $[\Delta_Y^{Q+1}, \Delta_U^{Q+1}, \Delta_V^{Q+1}]$, for all the channels). Unfortunately, this fact significantly complicates the optimal bit-rate control through quantization. 

A way to deal with this problem is ignore the orthogonality lack (well, the YCoCg transform is almost orthogonal, and therefore, we are not supposing something terribly wrong). This is that we are going to implement.

## RD curves of each YCrCb channel
Notice that we compute the distortion in the RGB domain because the transform is not orthogonal. The bit-rate is computed using only the corresponding channel. Remember that the quantization indexes images need to be normalized or equalized to be displayed properly.

In [ ]:
def Yrb_RD_curve(RGB_img, Q_steps):
    RD_points = []
    for q in range(Q_steps-1, -1, -1):
        YCrCb_img = YCrCb.from_RGB(RGB_img.astype(np.uint8))
        Y_img = YCrCb_img[:,:,0]
        dequantized_Y_img = np.empty_like(Y_img)
        Q_step = 1 << q
        k, dequantized_Y_img = q_deq(Y_img, Q_step)
        rate = bits_per_gray_pixel(k, 'Yrb' + str(Q_step) + '_')
        #_distortion = distortion.MSE(Y_img, dequantized_Y_img)
        YCrCb_img[...,0] = dequantized_Y_img
        dequantized_RGB_img = YCrCb.to_RGB(YCrCb_img)
        _distortion = distortion.MSE(RGB_img, dequantized_RGB_img)
        RD_points.append((rate, _distortion, 'Yrb', Q_step))
        print(f"q_step={Q_step:>3}, rate={rate:.3f} bits/pixel, distortion={_distortion}")
    return RD_points

def Cr_RD_curve(RGB_img, Q_steps):
    RD_points = []
    for q in range(Q_steps-1, -1, -1):
        YCrCb_img = YCrCb.from_RGB(RGB_img.astype(np.uint8))
        Cr_img = YCrCb_img[:,:,1]
        dequantized_Cr_img = np.empty_like(Cr_img)
        Q_step = 1 << q
        k, dequantized_Cr_img = q_deq(Cr_img, Q_step)
        rate = bits_per_gray_pixel(k, 'Cr' + str(Q_step) + '_')
        #_distortion = distortion.MSE(Cr_img, dequantized_Cr_img)
        YCrCb_img[...,1] = dequantized_Cr_img
        dequantized_RGB_img = YCrCb.to_RGB(YCrCb_img)
        _distortion = distortion.MSE(RGB_img, dequantized_RGB_img)
        RD_points.append((rate, _distortion, 'Cr', Q_step))
        print(f"q_step={Q_step:>3}, rate={rate:.3f} bits/pixel, distortion={_distortion}")
    return RD_points

def Cb_RD_curve(RGB_img, Q_steps):
    RD_points = []
    for q in range(Q_steps-1, -1, -1):
        YCrCb_img = YCrCb.from_RGB(RGB_img.astype(np.uint8))
        Cb_img = YCrCb_img[:,:,2]
        dequantized_Cb_img = np.empty_like(Cb_img)
        Q_step = 1 << q
        k, dequantized_Cb_img = q_deq(Cb_img, Q_step)
        rate = bits_per_gray_pixel(k, 'Cb' + str(Q_step) + '_')
        #_distortion = distortion.MSE(Cb_img, dequantized_Cb_img)
        YCrCb_img[...,2] = dequantized_Cb_img
        dequantized_RGB_img = YCrCb.to_RGB(YCrCb_img)
        _distortion = distortion.MSE(RGB_img, dequantized_RGB_img)        
        RD_points.append((rate, _distortion, 'Cb', Q_step))
        print(f"q_step={Q_step:>3}, rate={rate:.3f} bits/pixel, distortion={_distortion}")
    return RD_points

Yrb_points = Yrb_RD_curve(img, Q_steps)
Cr_points = Cr_RD_curve(img, Q_steps)
Cb_points = Cb_RD_curve(img, Q_steps)

In [ ]:
pylab.figure(dpi=150)
#pylab.plot(*zip(*YCrCb_points), c='m', marker="x",
#           label='$\Delta_{\mathrm{Y}} = \Delta_{\mathrm{Cr}} = \Delta_{\mathrm{Cb}}$')
pylab.plot(*zip(*[(i[0], i[1]) for i in Yrb_points]), c='r', marker="o", label='Only Y')              
pylab.plot(*zip(*[(i[0], i[1]) for i in Cr_points]), c='g', marker="o",
           label='Only Cr')              
pylab.plot(*zip(*[(i[0], i[1]) for i in Cb_points]), c='b', marker="o",
           label='Only Cb')              
pylab.title("RD Performance")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("MSE")
plt.legend(loc='upper right')
pylab.show()

Conclusions:
1. The distortions are not additive.
2. The slopes of the curves for different quantization steps are different.

### RD curves of each YCoCg channel

In [ ]:
def Yog_RD_curve(RGB_img, Q_steps):
    RD_points = []
    for q in range(Q_steps-1, -1, -1):
        YCoCg_img = YCoCg.from_RGB(RGB_img.astype(np.int16))
        Y_img = YCoCg_img[:,:,0]
        dequantized_Y_img = np.empty_like(Y_img)
        Q_step = 1 << q
        k, dequantized_Y_img = q_deq(Y_img, Q_step)
        k = k.astype(np.uint8)
        rate = bits_per_gray_pixel(k, 'Yog' + str(Q_step) + '_')
        #_distortion = distortion.MSE(Y_img, dequantized_Y_img)
        YCoCg_img[...,0] = dequantized_Y_img
        dequantized_RGB_img = YCoCg.to_RGB(YCoCg_img)
        _distortion = distortion.MSE(RGB_img, dequantized_RGB_img)        
        RD_points.append((rate, _distortion, 'Yog', Q_step))
        print(f"Q_step={Q_step:>3}, rate={rate:.3f} bits/pixel, distortion={_distortion}")
    return RD_points

def Co_RD_curve(RGB_img, Q_steps):
    RD_points = []
    for q in range(Q_steps-1, -1, -1):
        YCoCg_img = YCoCg.from_RGB(RGB_img.astype(np.int16))
        Co_img = YCoCg_img[:,:,1]
        dequantized_Co_img = np.empty_like(Co_img)
        Q_step = 1 << q
        k, dequantized_Co_img = q_deq(Co_img, Q_step)
        k = k.astype(np.uint8)
        rate = bits_per_gray_pixel(k, 'Co' + str(Q_step) + '_')
        #_distortion = distortion.MSE(Co_img, dequantized_Co_img)
        YCoCg_img[...,1] = dequantized_Co_img
        dequantized_RGB_img = YCoCg.to_RGB(YCoCg_img)
        _distortion = distortion.MSE(RGB_img, dequantized_RGB_img)        
        RD_points.append((rate, _distortion, 'Co', Q_step))
        print(f"Q_step={Q_step:>3}, rate={rate:.3f} bits/pixel, distortion={_distortion}")
    return RD_points

def Cg_RD_curve(RGB_img, Q_steps):
    RD_points = []
    for q in range(Q_steps-1, -1, -1):
        YCoCg_img = YCoCg.from_RGB(RGB_img.astype(np.int16))
        Cg_img = YCoCg_img[:,:,2]
        dequantized_Cg_img = np.empty_like(Cg_img)
        Q_step = 1 << q
        k, dequantized_Cg_img = q_deq(Cg_img, Q_step)
        k = k.astype(np.uint8)
        rate = bits_per_gray_pixel(k, 'Cg' + str(Q_step) + '_')
        #_distortion = distortion.MSE(Cg_img, dequantized_Cg_img)
        YCoCg_img[...,2] = dequantized_Cg_img
        dequantized_RGB_img = YCoCg.to_RGB(YCoCg_img)
        _distortion = distortion.MSE(RGB_img, dequantized_RGB_img)        
        RD_points.append((rate, _distortion, 'Cg', Q_step))
        print(f"Q_step={Q_step:>3}, rate={rate:.3f} bits/pixel, distortion={_distortion}")
    return RD_points

Yog_points = Yog_RD_curve(img, Q_steps)
Co_points = Co_RD_curve(img, Q_steps)
Cg_points = Cg_RD_curve(img, Q_steps)

In [ ]:
pylab.figure(dpi=150)
#pylab.plot(*zip(*YCoCg_points), c='m', marker="x",
#           label='$\Delta_{\mathrm{Y}} = \Delta_{\mathrm{Co}} = \Delta_{\mathrm{Cg}}$')
pylab.plot(*zip(*[(i[0], i[1]) for i in Yog_points]), c='r', marker="o", label='Only Y')              
pylab.plot(*zip(*[(i[0], i[1]) for i in Co_points]), c='g', marker="o",
           label='Only Co')              
pylab.plot(*zip(*[(i[0], i[1]) for i in Cg_points]), c='b', marker="o",
           label='Only Cb')              
pylab.title("RD Performance")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("MSE")
plt.legend(loc='upper right')
pylab.show()

Conclusions:
1. The distortions are not additive.
2. The slopes of the curves for different quantization steps are different.
3. The channel Y should be quantized less.

## Compute the slopes
In a different list for each component.

In [ ]:
def compute_slopes(RD_points):
    extended_RD_points = [(0.0, 0.0, '', -1)] + RD_points
    counter = 0
    RD_slopes = [(9.0E9, RD_points[0])]
    points_iterator = iter(RD_points)
    next(points_iterator)
    for i in points_iterator:
        BPP = i[0] # Rate 
        delta_BPP = BPP - RD_points[counter][0]
        MSE = i[1] # Distortion
        delta_MSE = MSE - RD_points[counter][1] 
        if delta_BPP > 0:
            slope = abs(delta_MSE/delta_BPP)
        else:
            slope = 0
        component = i[2]
        q_step = i[3]
        print((slope, i), delta_MSE, delta_BPP)
        RD_slopes.append((slope, i))
        counter += 1
    return RD_slopes

Yrb_slopes = compute_slopes(Yrb_points)
Cr_slopes = compute_slopes(Cr_points)
Cb_slopes = compute_slopes(Cb_points)

Yog_slopes = compute_slopes(Yog_points)
Co_slopes = compute_slopes(Co_points)
Cg_slopes = compute_slopes(Cg_points)

## Merge the RD slopes and sort them
By slope.

In [ ]:
all_YCrCb_slopes = Yrb_slopes + Cr_slopes + Cb_slopes
sorted_YCrCb_slopes = sorted(all_YCrCb_slopes, key=lambda x: x[0])[::-1]

all_YCoCg_slopes = Yog_slopes + Co_slopes + Cg_slopes
sorted_YCoCg_slopes = sorted(all_YCoCg_slopes, key=lambda x: x[0])[::-1]

## Build the sub-optimal RD curve

Progressively quantize the image using the Q_steps described in the sorted list of monotonously decreasing slopes, and then, compute the distortion and the bit-rate. Remember that the quantization indexes images must be normalized or equalized to be displayed properly, and also, that they are in the YUV domain.

In [ ]:
# Falta hacer la transformada!!

def YCrCb_get_suboptimal_RD_curve(RGB_img, sorted_slopes, components):
    points = []
    Q_steps_per_component = [256, 256, 256] # This should generate a black image.
    k = np.empty_like(RGB_img)
    y = np.empty_like(RGB_img)
    YCrCb_img = YCrCb.from_RGB(RGB_img.astype(np.uint8))
    for i in sorted_slopes:
        point = i[1]
        component = point[2]
        Q_step = point[3]
        Q_steps_per_component[components.index(component)] = Q_step
        #print(i, Q_steps_per_component)
        for c,Q_step in zip(components, Q_steps_per_component):
            k[..., components.index(c)], y[..., components.index(c)] = q_deq(YCrCb_img[..., components.index(c)], Q_step)
        k = k.astype(np.uint8)
        rate = bits_per_RGB_pixel(k, str(Q_steps_per_component))
        #_distortion = distortion.MSE(YCrCb_img, y)
        #YCoCg_img[...,2] = dequantized_Cg_img
        dequantized_RGB_img = YCrCb.to_RGB(y.astype(np.uint8))
        _distortion = distortion.MSE(RGB_img, dequantized_RGB_img)        
        points.append((rate, _distortion))
        print(f"Q_step={Q_steps_per_component}, rate={rate:>7} bits/pixel, distortion={_distortion:>6.1f}")
    return points

def YCoCg_get_suboptimal_RD_curve(RGB_img, sorted_slopes, components):
    points = []
    Q_steps_per_component = [256, 256, 256] # This should generate a black image.
    k = np.empty_like(RGB_img)
    y = np.empty_like(RGB_img)
    YCrCb_img = YCoCg.from_RGB(RGB_img.astype(np.int16))
    for i in sorted_slopes:
        point = i[1]
        component = point[2]
        Q_step = point[3]
        Q_steps_per_component[components.index(component)] = Q_step
        #print(i, Q_steps_per_component)
        for c,Q_step in zip(components, Q_steps_per_component):
            k[..., components.index(c)], y[..., components.index(c)] = q_deq(YCrCb_img[..., components.index(c)], Q_step)
        k = k.astype(np.uint8)
        rate = bits_per_RGB_pixel(k, str(Q_steps_per_component))
        #_distortion = distortion.MSE(YCrCb_img, y)
        #YCoCg_img[...,2] = dequantized_Cg_img
        dequantized_RGB_img = YCoCg.to_RGB(y)
        _distortion = distortion.MSE(RGB_img, dequantized_RGB_img)        
        points.append((rate, _distortion))
        print(f"Q_step={Q_steps_per_component}, rate={rate:>7} bits/pixel, distortion={_distortion:>6.1f}")
    return points

YCrCb_suboptimal_points = YCrCb_get_suboptimal_RD_curve(img, sorted_YCrCb_slopes, ['Yrb', 'Cr', 'Cb'])
YCoCg_suboptimal_points = YCoCg_get_suboptimal_RD_curve(img, sorted_YCoCg_slopes, ['Yog', 'Co', 'Cg'])

## Optimal quantization in the RGB domain
See [RGB_quantization.ipynb](file:///home/vruiz/Sistemas-Multimedia.github.io/milestones/05-RGB_quantization/RGB_quantization.ipynb).

In [ ]:
RGB_points = []
with open('../05-RGB_quantization/RGB.txt', 'r') as f:
    #rate, _distortion = f.read()
    #RGB_points.append((rate, _distortion))
    for line in f:
        rate, _distortion = line.split('\t')
        RGB_points.append((float(rate), float(_distortion)))

## RD curve using same $\Delta$ for each YUV channel ($\Delta_{\text{Y}} = \Delta_{\text{U}} = \Delta_{\text{V}}$)

In [ ]:
def YCrCb_RD_curve_same_delta(RGB_img):
    RD_points = []
    for q in range(0, 8):
        YCrCb_img = YCrCb.from_RGB(RGB_img.astype(np.uint8))
        #YCrCb_img = YCrCb_img.astype(np.int16)
        k, dequantized_YCrCb_img = q_deq(YCrCb_img, 1<<q)
        k = k.astype(np.uint8)
        #show(dequantized_YCrCb_img, q_step)
        rate = bits_per_RGB_pixel(k, "YCrCb" + str(1<<q) + "_")
        dequantized_RGB_img = YCrCb.to_RGB(dequantized_YCrCb_img.astype(np.uint8))
        _distortion = distortion.MSE(RGB_img, dequantized_RGB_img)
        RD_points.append((rate, _distortion))
        print(f"Q_step={1<<q:>3}, rate={rate:>7} bits/pixel, distortion={_distortion:>6.1f}")
    return RD_points

YCrCb_points_same_delta = YCrCb_RD_curve_same_delta(img)

In [ ]:
with open('YCrCb.txt', 'w') as f:
    for item in YCrCb_points_same_delta:
        f.write(f"{item[0]}\t{item[1]}\n")

In [ ]:
def YCoCg_RD_curve_same_delta(RGB_frame):
    RD_points = []
    for q in range(0, 8):
        YCoCg_frame = YCoCg.from_RGB(RGB_frame.astype(np.int16))
        k, dequantized_YCoCg_frame = q_deq(YCoCg_frame, 1<<q)
        k = k.astype(np.uint8)
        rate = bits_per_RGB_pixel(k, "YCoCg" + str(1<<q) + "_")
        dequantized_RGB_frame = YCoCg.to_RGB(dequantized_YCoCg_frame.astype(np.int16))
        _distortion = distortion.MSE(RGB_frame, dequantized_RGB_frame)
        RD_points.append((rate, _distortion))
        print(f"Q_step={1<<q:>3}, rate={rate:>7} bits/pixel, distortion={_distortion:>6.1f}")
    return RD_points

YCoCg_points_same_delta = YCoCg_RD_curve_same_delta(img)

In [ ]:
with open('YCoCg.txt', 'w') as f:
    for item in YCoCg_points_same_delta:
        f.write(f"{item[0]}\t{item[1]}\n")

## Let's compare!

In [ ]:
pylab.figure(dpi=150)
#pylab.plot(*zip(*RGB_points), marker="o", label="$\Delta_{\mathrm{R}}=\Delta_{\mathrm{G}}=\Delta_{\mathrm{B}}}$")
pylab.plot(*zip(*YCrCb_points_same_delta), marker="o", label="$\Delta_{\mathrm{Y}}=\Delta_{\mathrm{Cr}}=\Delta_{\mathrm{Cb}}}$")
pylab.plot(*zip(*YCoCg_points_same_delta), marker="o", label="$\Delta_{\mathrm{Y}}=\Delta_{\mathrm{Co}}=\Delta_{\mathrm{Cg}}}$")
pylab.plot(*zip(*YCrCb_suboptimal_points), marker="o", label="Suboptimal YCrCb")
pylab.plot(*zip(*YCoCg_suboptimal_points), marker="o", label="Suboptimal YCoCg")
pylab.title("Rate/Distortion Performance ")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("MSE")
pylab.legend(loc='upper right')
pylab.show()

## Conclusions:
1. In general, quantization is more effective in the transformed domain.
2. At low bit-rates, it's better to quantize in YCoCb domain than to quantize in the YCrCb one. 

### RD using channel gains in YCoCg domain

In [ ]:
def YCoCg_RD_curve(RGB_img):
    RD_points = []
    #for q_step in range(0, 8):
    for Q_step in range(1, 256, 16):
        YCoCg_img = YCoCg.from_RGB(RGB_img.astype(np.int16))
        #k, dequantized_YCoCg_img = q_deq(YCoCg_img, 1<<q_step)
        k, dequantized_YCoCg_img = q_deq(YCoCg_img, Q_step)
        k = k.astype(np.uint8)
        rate = bits_per_RGB_pixel(k)
        dequantized_RGB_img = YCoCg.to_RGB(dequantized_YCoCg_img)
        _distortion = distortion.MSE(RGB_img, dequantized_RGB_img)
        RD_points.append((rate, _distortion))
        print(f"Q_step={Q_step:>3}, rate={rate:>7} BPS, distortion={_distortion:>6.1f}")
    return RD_points

YCoCg_points = YCoCg_RD_curve(img)

relative_Y_gain = 3/2
relative_Co_gain = 1
relative_Cg_gain = 3/2
def YCoCg_RD_curve_with_gains(RGB_img):
    RD_points = []
    #for q_step in range(0, 8):
    for Q_step in range(1, 256, 16):
        YCoCg_img = YCoCg.from_RGB(RGB_img.astype(np.int16))
        dequantized_YCoCg_img = np.empty_like(YCoCg_img)
        k = np.empty_like(YCoCg_img, dtype=np.uint8)
        k[:,:,0], dequantized_YCoCg_img[:,:,0] = q_deq(YCoCg_img[:,:,0], (Q_step)/relative_Y_gain)
        k[:,:,1], dequantized_YCoCg_img[:,:,1] = q_deq(YCoCg_img[:,:,1], (Q_step)/relative_Co_gain)
        k[:,:,2], dequantized_YCoCg_img[:,:,2] = q_deq(YCoCg_img[:,:,2], (Q_step)/relative_Cg_gain)
        #k[0], dequantized_YCoCg_img[0] = q_deq(YCoCg_img[0], q_step/relative_Y_gain)
        #k[1], dequantized_YCoCg_img[1] = q_deq(YCoCg_img[1], q_step/relative_Co_gain)
        #k[2], dequantized_YCoCg_img[2] = q_deq(YCoCg_img[2], q_step/relative_Cg_gain)
        rate = bits_per_RGB_pixel(k)
        dequantized_RGB_img = YCoCg.to_RGB(dequantized_YCoCg_img)
        _distortion = distortion.MSE(RGB_img, dequantized_RGB_img)
        RD_points.append((rate, _distortion))
        print(f"Q_step={Q_step:>3}, rate={rate:>7} BPS, distortion={_distortion:>6.1f}")
    return RD_points

YCoCg_gains_points = YCoCg_RD_curve_with_gains(img)

In [ ]:
pylab.figure(dpi=150)
pylab.scatter(*zip(*YCoCg_points), c='r', marker=".", s=0.5,
           label='$\Delta_{\mathrm{Y}}=\Delta_{\mathrm{Co}}=\Delta_{\mathrm{Cg}}$')
pylab.scatter(*zip(*YCoCg_gains_points), c='b', marker="x", s=1,
           label='$\Delta_{\mathrm{Y}}=' + "{:3.1f}".format(relative_Y_gain) + '\Delta_{\mathrm{Cg}}' +
           ';\Delta_{\mathrm{Cg}}=' + "{:3.1f}".format(relative_Cg_gain) + '\Delta_{\mathrm{Co}}$')
pylab.title("Performance of Quantization in Different Domains")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("MSE")
plt.legend(loc='upper right')
pylab.show()

### Comparing the three domains using the same quantization steps

In [ ]:
pylab.figure(dpi=150)
#pylab.plot(*zip(*RGB_points), c='b', marker="x",
#           label='$\Delta_{\mathrm{R}}=\Delta_{\mathrm{G}}=\Delta_{\mathrm{B}}$')
#pylab.plot(*zip(*YCrCb_points), c='r', marker="x",
#           label='$\Delta_{\mathrm{Y}}=\Delta_{\mathrm{Cr}}=\Delta_{\mathrm{Cb}}$')
#pylab.plot(*zip(*YYCrCb_gains_points), c='m', marker="x",
#           label='$\Delta_{\mathrm{Y}}=' + "{:3.1f}".format(Y_gain) + '\Delta_{\mathrm{Cr}}' +
#           ';\Delta_{\mathrm{Cb}}=' + "{:3.1f}".format(Cb_gain) + '\Delta_{\mathrm{Cr}}$')
pylab.plot(*zip(*YCoCg_points), c='g', marker="x",
           label='$\Delta_{\mathrm{Y}}=\Delta_{\mathrm{Co}}=\Delta_{\mathrm{Cg}}$')
pylab.title("Performance of Quantization in Different Domains")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("MSE")
plt.legend(loc='upper right')
pylab.show()

In [ ]:
Cr_gain = 1.0 # 2.4754
Cb_gain = 3.25832/2.4754
Y_gain = 3/2.4754
def YYCrCb_RD_curve_with_gains(RGB_frame):
    RD_points = []
    for q_step in range(0, 8):
        YYCrCb_frame = YCrCb.from_RGB(RGB_frame.astype(np.uint8))
        dequantized_YYCrCb_frame = np.empty_like(YYCrCb_frame)
        k = np.empty_like(YYCrCb_frame, dtype=np.uint8)
        k[:,:,0], dequantized_YYCrCb_frame[:,:,0] = q_deq(YYCrCb_frame[:,:,0], (1<<q_step)/Y_gain)
        k[:,:,1], dequantized_YYCrCb_frame[:,:,1] = q_deq(YYCrCb_frame[:,:,1], (1<<q_step)/Cr_gain)
        k[:,:,2], dequantized_YYCrCb_frame[:,:,2] = q_deq(YYCrCb_frame[:,:,2], (1<<q_step)/Cb_gain)
        rate = bits_per_RGB_pixel(k)
        dequantized_YYCrCb_frame = dequantized_YYCrCb_frame.astype(np.uint8)
        dequantized_RGB_frame = YCrCb.to_RGB(dequantized_YYCrCb_frame)
        _distortion = distortion.MSE(RGB_frame, dequantized_RGB_frame)
        RD_points.append((rate, _distortion))
        print(f"q_step={1<<q_step:>3}, rate={rate:>7} BPS, distortion={_distortion:>6.1f}")
    return RD_points

YYCrCb_gains_points = YYCrCb_RD_curve_with_gains(img)

Conclusions:
1. In general, quantization is more effective in the transformed domain considering the RD plane.
2. $\Delta_{\mathrm{Y}}=1.2\Delta_{\mathrm{Cr}}; \Delta_{\mathrm{Cb}}=1.3\Delta_{\mathrm{Cr}}$ is slightly better than $\Delta_{\mathrm{Y}}=\Delta_{\mathrm{Cr}}=\Delta_{\mathrm{Cb}}$, at low bit-rates, and viceversa.
3. At low bit-rates, tt's better to quantize YCoCb than to quantize YCrCb. 

## Is $\Delta_{\mathrm{Y}}=1.2\Delta_{\mathrm{Cr}}; \Delta_{\mathrm{Cb}}=1.3\Delta_{\mathrm{Cr}}$ the best quantization in YCrCb?
No, $\Delta_{\mathrm{Y}}=\Delta_{\mathrm{Cr}}=\Delta_{\mathrm{Cb}}$ is better than $\Delta_{\mathrm{Y}}=1.2\Delta_{\mathrm{Cr}}; \Delta_{\mathrm{Cb}}=1.3\Delta_{\mathrm{Cr}}$ at high bit-rates.

## Is $\Delta_{\mathrm{Y}}=1.2\Delta_{\mathrm{Cr}}; \Delta_{\mathrm{Cb}}=1.3\Delta_{\mathrm{Cr}}$ optimal at low bit-rates quantizing YCrCb?

In [ ]:
N=5
def only_Y_RD_curve(RGB_frame):
    RD_points = []
    for q_step in range(0, 8):
        print(q_step, end=' ')
        YCrCb_frame = YCrCb.from_RGB(RGB_frame.astype(np.uint8))
        dequantized_YCrCb_frame = np.empty_like(YCrCb_frame)
        k = np.empty_like(YCrCb_frame, dtype=np.uint8)
        k[:,:,0], dequantized_YCrCb_frame[:,:,0] = q_deq(YCrCb_frame[:,:,0], 1<<q_step)
        k[:,:,1], dequantized_YCrCb_frame[:,:,1] = q_deq(YCrCb_frame[:,:,1], 1<<N)
        k[:,:,2], dequantized_YCrCb_frame[:,:,2] = q_deq(YCrCb_frame[:,:,2], 1<<N)
        rate = bits_per_RGB_pixel(k)
        dequantized_YCrCb_frame = dequantized_YCrCb_frame.astype(np.uint8)
        assert dequantized_YCrCb_frame.all() >= 0
        dequantized_RGB_frame = YCrCb.to_RGB(dequantized_YCrCb_frame)
        _distortion = distortion.MSE(RGB_frame, dequantized_RGB_frame)
        RD_points.append((rate, _distortion))
    return RD_points

def only_Cb_RD_curve(RGB_frame):
    RD_points = []
    for q_step in range(0, 8):
        print(q_step, end=' ')
        YCrCb_frame = YCrCb.from_RGB(RGB_frame.astype(np.uint8))
        dequantized_YCrCb_frame = np.empty_like(YCrCb_frame)
        k = np.empty_like(YCrCb_frame, dtype=np.uint8)
        k[:,:,0], dequantized_YCrCb_frame[:,:,0] = q_deq(YCrCb_frame[:,:,0], 1<<N)
        k[:,:,1], dequantized_YCrCb_frame[:,:,1] = q_deq(YCrCb_frame[:,:,1], 1<<q_step)
        k[:,:,2], dequantized_YCrCb_frame[:,:,2] = q_deq(YCrCb_frame[:,:,2], 1<<N)
        rate = bits_per_RGB_pixel(k)
        assert dequantized_YCrCb_frame.all() >= 0
        dequantized_YCrCb_frame = dequantized_YCrCb_frame.astype(np.uint8)
        dequantized_RGB_frame = YCrCb.to_RGB(dequantized_YCrCb_frame)
        _distortion = distortion.MSE(RGB_frame, dequantized_RGB_frame)
        RD_points.append((rate, _distortion))
    return RD_points

def only_Cr_RD_curve(RGB_frame):
    RD_points = []
    for q_step in range(0, 8):
        print(q_step, end=' ')
        YCrCb_frame = YCrCb.from_RGB(RGB_frame.astype(np.uint8))
        dequantized_YCrCb_frame = np.empty_like(YCrCb_frame)
        k = np.empty_like(YCrCb_frame, dtype=np.uint8)
        k[:,:,0], dequantized_YCrCb_frame[:,:,0] = q_deq(YCrCb_frame[:,:,0], 1<<N)
        k[:,:,1], dequantized_YCrCb_frame[:,:,1] = q_deq(YCrCb_frame[:,:,1], 1<<N)
        k[:,:,2], dequantized_YCrCb_frame[:,:,2] = q_deq(YCrCb_frame[:,:,2], 1<<q_step)
        rate = bits_per_RGB_pixel(k)
        dequantized_YCrCb_frame = dequantized_YCrCb_frame.astype(np.uint8)
        assert dequantized_YCrCb_frame.all() >= 0
        dequantized_RGB_frame = YCrCb.to_RGB(dequantized_YCrCb_frame)
        _distortion = distortion.MSE(RGB_frame, dequantized_RGB_frame)
        RD_points.append((rate, _distortion))
    return RD_points

only_Y_points = only_Y_RD_curve(img)
only_Cb_points = only_Cb_RD_curve(img)
only_Cr_points = only_Cr_RD_curve(img)

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*YYCrCb_gains_points), c='m', marker="x",
           label='$\Delta_{\mathrm{Y}}=' + "{:3.1f}".format(Y_gain) + '\Delta_{\mathrm{Cr}}' +
           ';\Delta_{\mathrm{Cb}}=' + "{:3.1f}".format(Cb_gain) + '\Delta_{\mathrm{Cr}}$')
pylab.plot(*zip(*only_Y_points), c='r', marker="o",
           label='$\Delta_{\mathrm{Y}}~\mathrm{varies},~\Delta_{\mathrm{Cr}}=\Delta_{\mathrm{Cb}}=$' + '{}'.format(1<<N))              
pylab.plot(*zip(*only_Cb_points), c='g', marker="o",
           label='$\Delta_{\mathrm{Cb}}~\mathrm{varies},~\Delta_{\mathrm{Y}}=\Delta_{\mathrm{Cr}}=$' + '{}'.format(1<<N))              
pylab.plot(*zip(*only_Cr_points), c='b', marker="o",
           label='$\Delta_{\mathrm{Cr}}~\mathrm{varies},~\Delta_{\mathrm{Y}}=\Delta_{\mathrm{Cb}}=$' + '{}'.format(1<<N))              
pylab.title("RD Performance")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("RMSE")
plt.legend(loc='upper right')
pylab.show()

No, there are combinations of $\Delta_{\mathrm{Y}}$, $\Delta_{\mathrm{Cr}}$, and $\Delta_{\mathrm{Cb}}$ better than $\Delta_{\mathrm{Y}}=1.2\Delta_{\mathrm{Cr}}; \Delta_{\mathrm{Cb}}=1.3\Delta_{\mathrm{Cr}}$ at low bit-rates.

## Is $\Delta_{\mathrm{Y}}=\Delta_{\mathrm{Co}}=\Delta_{\mathrm{Cg}}$ optimal quantizing YCoCb?

In [ ]:
N=4
def only_Y_RD_curve(RGB_frame):
    RD_points = []
    for q_step in range(0, 8):
        print(q_step, end=' ')
        YCoCg_frame = YCoCg.from_RGB(RGB_frame.astype(np.int16))
        dequantized_YCoCg_frame = np.empty_like(YCoCg_frame)
        k = np.empty_like(YCoCg_frame, dtype=np.uint8)
        k[:,:,0], dequantized_YCoCg_frame[:,:,0] = q_deq(YCoCg_frame[:,:,0], 1<<q_step)
        k[:,:,1], dequantized_YCoCg_frame[:,:,1] = q_deq(YCoCg_frame[:,:,1], 1<<N)
        k[:,:,2], dequantized_YCoCg_frame[:,:,2] = q_deq(YCoCg_frame[:,:,2], 1<<N)
        #k[:,:,1], dequantized_YCoCg_frame[:,:,1] = q_deq(YCoCg_frame[:,:,1], 1<<q_step)
        #k[:,:,2], dequantized_YCoCg_frame[:,:,2] = q_deq(YCoCg_frame[:,:,2], 1<<q_step)
        rate = bits_per_RGB_pixel(k)
        dequantized_RGB_frame = YCoCg.to_RGB(dequantized_YCoCg_frame)
        _distortion = distortion.MSE(RGB_frame, dequantized_RGB_frame)
        RD_points.append((rate, _distortion))
    return RD_points

def only_Co_RD_curve(RGB_frame):
    RD_points = []
    for q_step in range(0, 8):
        print(q_step, end=' ')
        YCoCg_frame = YCoCg.from_RGB(RGB_frame.astype(np.int16))
        dequantized_YCoCg_frame = np.empty_like(YCoCg_frame)
        k = np.empty_like(YCoCg_frame, dtype=np.uint8)
        k[:,:,0], dequantized_YCoCg_frame[:,:,0] = q_deq(YCoCg_frame[:,:,0], 1<<N)
        k[:,:,1], dequantized_YCoCg_frame[:,:,1] = q_deq(YCoCg_frame[:,:,1], 1<<q_step)
        k[:,:,2], dequantized_YCoCg_frame[:,:,2] = q_deq(YCoCg_frame[:,:,2], 1<<N)
        rate = bits_per_RGB_pixel(k)
        dequantized_RGB_frame = YCoCg.to_RGB(dequantized_YCoCg_frame)
        _distortion = distortion.MSE(RGB_frame, dequantized_RGB_frame)
        RD_points.append((rate, _distortion))
    return RD_points

def only_Cg_RD_curve(RGB_frame):
    RD_points = []
    for q_step in range(0, 8):
        print(q_step, end=' ')
        YCoCg_frame = YCoCg.from_RGB(RGB_frame.astype(np.int16))
        dequantized_YCoCg_frame = np.empty_like(YCoCg_frame)
        k = np.empty_like(YCoCg_frame, dtype=np.uint8)
        k[:,:,0], dequantized_YCoCg_frame[:,:,0] = q_deq(YCoCg_frame[:,:,0], 1<<N)
        k[:,:,1], dequantized_YCoCg_frame[:,:,1] = q_deq(YCoCg_frame[:,:,1], 1<<N)
        k[:,:,2], dequantized_YCoCg_frame[:,:,2] = q_deq(YCoCg_frame[:,:,2], 1<<q_step)
        rate = bits_per_RGB_pixel(k)
        dequantized_RGB_frame = YCoCg.to_RGB(dequantized_YCoCg_frame)
        _distortion = distortion.MSE(RGB_frame, dequantized_RGB_frame)
        RD_points.append((rate, _distortion))
    return RD_points

only_Y_points = only_Y_RD_curve(img)
only_Co_points = only_Co_RD_curve(img)
only_Cg_points = only_Cg_RD_curve(img)

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*only_Y_points), c='r', marker="o",
           label='$\Delta_{\mathrm{Y}}~\mathrm{varies},~\Delta_{\mathrm{Co}}=\Delta_{\mathrm{Cg}}=$' + '{}'.format(1<<N))              
pylab.plot(*zip(*only_Co_points), c='m', marker="o",
           label='$\Delta_{\mathrm{Co}}~\mathrm{varies},~\Delta_{\mathrm{Y}}=\Delta_{\mathrm{Cg}}=$' + '{}'.format(1<<N))              
pylab.plot(*zip(*only_Cg_points), c='b', marker="o",
           label='$\Delta_{\mathrm{Cg}}~\mathrm{varies},~\Delta_{\mathrm{Y}}=\Delta_{\mathrm{Co}}=$' + '{}'.format(1<<N))              
pylab.plot(*zip(*YCoCg_points), c='g', marker="x",
           label='$\Delta_{\mathrm{Y}}=\Delta_{\mathrm{Co}}=\Delta_{\mathrm{Cg}}$')
pylab.title("RD Performance")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("RMSE")
plt.legend(loc='upper right')
pylab.show()

At least, using the same experiment than before, $\Delta_{\mathrm{Y}}=\Delta_{\mathrm{Co}}=\Delta_{\mathrm{Cg}}$ seems to be near optimal quantizing YCoCb.

## Ignore the rest ...

In [ ]:
input()

## Some experiments showing the impact of the lack of orthogonality

In [ ]:
def _YCbCr_RD_curve(RGB_frame, N):
    RD_points = []
    for q_step in range(0, 8):
        print(q_step, end=' ')
        YCbCr_frame = RGB_to_YCbCr(RGB_frame.astype(np.uint8))
        dequantized_YCbCr_frame = np.empty_like(YCbCr_frame)
        k = np.empty_like(YCbCr_frame)
        k[:,:,0], dequantized_YCbCr_frame[:,:,0] = q_deq(YCbCr_frame[:,:,0], 1<<q_step)
        k[:,:,1], dequantized_YCbCr_frame[:,:,1] = q_deq(YCbCr_frame[:,:,1], 1<<N)
        k[:,:,2], dequantized_YCbCr_frame[:,:,2] = q_deq(YCbCr_frame[:,:,2], 1<<N)
        rate = bytes_per_color_frame(k)
        dequantized_YCbCr_frame = dequantized_YCbCr_frame.astype(np.uint8)
        dequantized_RGB_frame = YCbCr_to_RGB(dequantized_YCbCr_frame)
        distortion = RMSE(RGB_frame, dequantized_RGB_frame)
        RD_points.append((rate, distortion))
    return RD_points

_YCbCr_points_8 = _YCbCr_RD_curve(frame, 8)
_YCbCr_points_7 = _YCbCr_RD_curve(frame, 7)
_YCbCr_points_6 = _YCbCr_RD_curve(frame, 6)
_YCbCr_points_5 = _YCbCr_RD_curve(frame, 5)
_YCbCr_points_4 = _YCbCr_RD_curve(frame, 4)
_YCbCr_points_3 = _YCbCr_RD_curve(frame, 3)

In [ ]:
1<<4

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*YCbCr_points), c='r', marker="x",
           label='$\Delta_{\mathrm{Y}}=\Delta_{\mathrm{Cr}}=\Delta_{\mathrm{Cb}}$')
pylab.plot(*zip(*_YCbCr_points_8), c='b', marker="x",
           label='$\Delta_{\mathrm{Y}}~\mathrm{varies},~\Delta_{\mathrm{Cr}}=\Delta_{\mathrm{Cb}}=$' + '{}'.format(1<<8))
pylab.plot(*zip(*_YCbCr_points_7), c='g', marker="x",
           label='$\Delta_{\mathrm{Y}}~\mathrm{varies},~\Delta_{\mathrm{Cr}}=\Delta_{\mathrm{Cb}}=$' + '{}'.format(1<<7))
pylab.plot(*zip(*_YCbCr_points_6), c='c', marker="x",
           label='$\Delta_{\mathrm{Y}}~\mathrm{varies},~\Delta_{\mathrm{Cr}}=\Delta_{\mathrm{Cb}}=$' + '{}'.format(1<<6))
pylab.plot(*zip(*_YCbCr_points_5), c='m', marker="x",
           label='$\Delta_{\mathrm{Y}}~\mathrm{varies},~\Delta_{\mathrm{Cr}}=\Delta_{\mathrm{Cb}}=$' + '{}'.format(1<<5))
pylab.plot(*zip(*_YCbCr_points_4), c='y', marker="x",
           label='$\Delta_{\mathrm{Y}}~\mathrm{varies},~\Delta_{\mathrm{Cr}}=\Delta_{\mathrm{Cb}}=$' + '{}'.format(1<<4))
pylab.plot(*zip(*_YCbCr_points_3), c='k', marker="o",
           label='$\Delta_{\mathrm{Y}}~\mathrm{varies},~\Delta_{\mathrm{Cr}}=\Delta_{\mathrm{Cb}}=$' + '{}'.format(1<<3))
pylab.plot(*zip(*YCbCr_gains_points), c='m', marker="+",
           label='$\Delta_{\mathrm{Y}}=' + "{:3.1f}".format(Y_gain) + '\Delta_{\mathrm{Cr}}' +
           ';\Delta_{\mathrm{Cb}}=' + "{:3.1f}".format(Cb_gain) + '\Delta_{\mathrm{Cr}}$')
pylab.title("The lack of non-orthogonality in the YCrCb transform")
pylab.xlabel("Bytes/Frame")
pylab.ylabel("RMSE")
plt.legend(loc='upper right')
pylab.show()

From this experiment we conclude that:
1. The luma should not be "deleted" from the code-stream (see curve $\Delta_{\mathrm{Y}}=\Delta_{\mathrm{Cr}}=\Delta_{\mathrm{Cb}}$).
2. There are better combinations than $\Delta_{\mathrm{Y}}=\Delta_{\mathrm{Cr}}=\Delta_{\mathrm{Cb}}$ and $\Delta_{\mathrm{Y}}=1.2\Delta_{\mathrm{Cr}};\Delta_{\mathrm{Cb}}=1.3\Delta_{\mathrm{Cr}}$.

### YCrCb

It's possible to find better combinations than $\Delta_{\mathrm{Y}}=1.2\Delta_{\mathrm{Cr}};\Delta_{\mathrm{Cb}}=1.3\Delta_{\mathrm{Cr}}$.

Notice that, at least visually, it does not make sense to use $\Delta_{\mathrm{Y}}\ge 8$ because the reconstructed image will be very dark or even black. The same holds for YCoCg.

In [ ]:
ycc = RGB_to_YCbCr(frame.astype(np.uint8))
ycc[:,:,0] = 0
frame2 = YCbCr_to_RGB(ycc)
print(frame2.min(), frame2.max())
show_frame(frame2, "$\Delta_{\mathrm{Y}} \ge 8$" + " (YCbCr domain)")

### YCoCg

Notice that, at least visually, it does not make sense to use $\Delta_{\mathrm{Y}}\ge 8$ because the reconstructed image will be very dark or even black. The same holds for YCoCg.

In [ ]:
ycc = RGB_to_YCoCg(frame)
ycc[:,:,0]= 0
frame2 = YCoCg_to_RGB(ycc)
print(frame2.min(), frame2.max())
show_frame(frame2, "$\Delta_{\mathrm{Y}} \ge 8$" + " (YCoCg domain)")

In [ ]:
def YCbCr_RD_curve_only_Y(RGB_frame):
    RD_points = []
    for q_step in range(0, 8):
        print(q_step, end=' ')
        YCbCr_frame = RGB_to_YCbCr(RGB_frame.astype(np.uint8))
        YCbCr_frame[:,:,1] = 0
        YCbCr_frame[:,:,2] = 0
        k, dequantized_YCbCr_frame = q_deq(YCbCr_frame, 1<<q_step)
        k[:,:,0] = 0
        rate = byte_rate(k)
        dequantized_YCbCr_frame = dequantized_YCbCr_frame.astype(np.uint8)
        dequantized_RGB_frame = YCbCr_to_RGB(dequantized_YCbCr_frame)
        distortion = RMSE(RGB_frame, dequantized_RGB_frame)
        RD_points.append((rate, distortion))
    return RD_points

def YCbCr_RD_curve_only_Cb(RGB_frame):
    RD_points = []
    for q_step in range(0, 8):
        print(q_step, end=' ')
        YCbCr_frame = RGB_to_YCbCr(RGB_frame.astype(np.uint8))
        YCbCr_frame[:,:,0] = 0
        YCbCr_frame[:,:,2] = 0
        k, dequantized_YCbCr_frame = q_deq(YCbCr_frame, 1<<q_step)
        rate = byte_rate(k)
        dequantized_YCbCr_frame = dequantized_YCbCr_frame.astype(np.uint8)
        dequantized_RGB_frame = YCbCr_to_RGB(dequantized_YCbCr_frame)
        distortion = RMSE(RGB_frame, dequantized_RGB_frame)
        RD_points.append((rate, distortion))
    return RD_points

def YCbCr_RD_curve_only_Cr(RGB_frame):
    RD_points = []
    for q_step in range(0, 8):
        print(q_step, end=' ')
        YCbCr_frame = RGB_to_YCbCr(RGB_frame.astype(np.uint8))
        YCbCr_frame[:,:,0] = 0
        YCbCr_frame[:,:,1] = 0
        k, dequantized_YCbCr_frame = q_deq(YCbCr_frame, 1<<q_step)
        rate = byte_rate(k)
        dequantized_YCbCr_frame = dequantized_YCbCr_frame.astype(np.uint8)
        dequantized_RGB_frame = YCbCr_to_RGB(dequantized_YCbCr_frame)
        distortion = RMSE(RGB_frame, dequantized_RGB_frame)
        RD_points.append((rate, distortion))
    return RD_points

only_Y_curve = YCbCr_RD_curve_only_Y(frame)
only_Cb_curve = YCbCr_RD_curve_only_Cb(frame)
only_Cr_curve = YCbCr_RD_curve_only_Cr(frame)

In [ ]:
pylab.figure(dpi=150)
pylab.scatter(*zip(*only_Y_curve), s=2, c='r', marker="o", label='only Y')
pylab.plot(*zip(*only_Y_curve), c='r', marker="o")
pylab.scatter(*zip(*only_Cb_curve), s=2, c='g', marker="o", label='only Cb')
pylab.plot(*zip(*only_Cb_curve), c='g', marker="o")
pylab.scatter(*zip(*only_Cr_curve), s=2, c='b', marker="o", label='only Cr')
pylab.plot(*zip(*only_Cr_curve), c='b', marker="o")
pylab.title("R/D Performance")
pylab.xlabel("Bytes/Frame")
pylab.ylabel("RMSE")
plt.legend(loc='upper right')
pylab.show()

In [ ]:
def YCbCr_RD_curve_2(RGB_frame):
    RD_points = []
    for q_step in range(0, 8):
        print(q_step, end=' ')
        YCbCr_frame = RGB_to_YCbCr(RGB_frame.astype(np.uint16))
        YCbCr_frame[:,:,1] //= 2
        YCbCr_frame[:,:,2] //= 2
        k, dequantized_YCbCr_frame = q_deq(YCbCr_frame, 1<<q_step)
        dequantized_YCbCr_frame[:,:,1] *= 2
        dequantized_YCbCr_frame[:,:,2] *= 2
        rate = byte_rate(k)
        dequantized_YCbCr_frame = dequantized_YCbCr_frame.astype(np.uint16)
        dequantized_RGB_frame = YCbCr_to_RGB(dequantized_YCbCr_frame)
        distortion = RMSE(RGB_frame, dequantized_RGB_frame)
        RD_points.append((rate, distortion))
    return RD_points

YCbCr_quantization_2 = YCbCr_RD_curve_2(frame)

In [ ]:
pylab.figure(dpi=150)
pylab.scatter(*zip(*RGB_quantization), s=2, c='b', marker="o", label='RGB quantization')
pylab.plot(*zip(*RGB_quantization), c='b', marker="o")
pylab.scatter(*zip(*YCbCr_quantization), s=2, c='r', marker="o", label='YCbCr quantization')
pylab.plot(*zip(*YCbCr_quantization), c='r', marker="o")
pylab.scatter(*zip(*YCbCr_quantization_2), s=2, c='g', marker="o", label='YCbCr quantization 2')
pylab.plot(*zip(*YCbCr_quantization_2), c='g', marker="o")
pylab.title("R/D Only Quantization")
pylab.xlabel("Bytes/Frame")
pylab.ylabel("RMSE")
plt.legend(loc='upper right')
pylab.show()

In [ ]:
YCbCr_test_frame = np.array([255, 0, 0], dtype=np.int16).reshape((1,1,1))
print(YCbCr_to_RGB(YCbCr_test_frame))

In [ ]:
np.array([255, 0, 0], dtype=np.int16)

In [ ]:
YCbCr_test_frame = np.zeros_like(frame).astype(np.uint16)

In [ ]:
type(YCbCr_test_frame[0,0,0])

In [ ]:
YCbCr_test_frame[1,1,2] = 255

In [ ]:
RGB_test_frame = YCbCr_to_RGB(YCbCr_test_frame)

In [ ]:
print(average_energy(RGB_test_frame))

In [ ]:
show_frame(RGB_test_frame, "")